In [1]:
import numpy as np
import struct
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as Data
import torchvision
from copy import deepcopy
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import time
from scipy import spatial
from util.util import mnist_noise

from trajectoryReweight.model import WeightedCrossEntropyLoss, TrajectoryReweightNN
from trajectoryReweight.baseline import StandardTrainingNN
from trajectoryReweight.gmm import GaussianMixture

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

def read_idx(filename):
    with open(filename, 'rb') as f:
        zero, data_type, dims = struct.unpack('>HBB', f.read(4))
        shape = tuple(struct.unpack('>I', f.read(4))[0] for d in range(dims))
        return np.frombuffer(f.read(), dtype=np.uint8).reshape(shape)

cuda:0


In [2]:
"""
CNN
"""
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(7 * 7 * 64, 1000)
        self.fc2 = nn.Linear(1000, 10)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out
    
def accuracy(predict_y, test_y):
    score = 0
    for pred, acc in zip(predict_y, test_y):
        if pred == acc:
            score +=1
    return score / test_y.shape[0]

In [ ]:
"""
MNIST DATA
"""
x_train = read_idx('data/train-images.idx3-ubyte')
y_train = read_idx('data/train-labels.idx1-ubyte')
x_test = read_idx('data/t10k-images.idx3-ubyte')
y_test = read_idx('data/t10k-labels.idx1-ubyte')
valid_idx = np.random.choice(range(60000), size=1000, replace=False)
x_valid = x_train[valid_idx]
y_valid = y_train[valid_idx]
x_train = np.delete(x_train, valid_idx, axis=0)
y_train = np.delete(y_train, valid_idx)

"""
shrink dataset to make noisy significant
"""
subset_idx = np.random.choice(range(60000-1000), size=10000, replace=False)
x_train = x_train[subset_idx]
y_train = y_train[subset_idx]

"""
Add Noise label to training data
"""
y_train_noisy = mnist_noise(y_train,0.1)

In [ ]:
"""
Initializaion
"""
train_idx = np.arange(len(x_train))
x_train = np.transpose(x_train,(2,1,0))
x_valid = np.transpose(x_valid,(2,1,0))
x_test = np.transpose(x_test,(2,1,0))
x_train_tensor = torchvision.transforms.ToTensor()(x_train).unsqueeze(1)
x_valid_tensor = torchvision.transforms.ToTensor()(x_valid).unsqueeze(1)
x_test_tensor = torchvision.transforms.ToTensor()(x_test).unsqueeze(1)
y_train_tensor = torch.from_numpy(y_train.astype(np.long))
y_train_noisy_tensor = torch.from_numpy(y_train_noisy.astype(np.long))
y_valid_tensor = torch.from_numpy(y_valid.astype(np.long))
y_test_tensor = torch.from_numpy(y_test.astype(np.long))

In [ ]:
"""
CNN without reweight
"""
cnn = CNN()
cnn.to(device)

stand_trainNN = StandardTrainingNN(cnn,
                                   batch_size=64,
                                   num_iter=80,
                                   learning_rate=1e-3,
                                   early_stopping=5,
                                   device=device,
                                   iprint=1)
stand_trainNN.fit(x_train_tensor, y_train_noisy_tensor, x_valid_tensor, y_valid_tensor, x_test_tensor, y_test_tensor)

test_output_y = stand_trainNN.predict(x_test_tensor)
test_accuracy = accuracy(test_output_y, y_test_tensor.data.numpy())

print('test accuracy is {}%'.format(100 * test_accuracy))

In [ ]:
"""
CNN with reweight
"""
cnn = CNN()
cnn.to(device)

tra_weightNN = TrajectoryReweightNN(cnn,
                                    burnin=5,
                                    num_cluster=5,
                                    batch_size=64,
                                    num_iter=80,
                                    learning_rate=1e-3,
                                    early_stopping=5,
                                    device=device,
                                    traj_step = 3,
                                    iprint=1)
tra_weightNN.fit(x_train_tensor, y_train_noisy_tensor, x_valid_tensor, y_valid_tensor,x_test_tensor, y_test_tensor)

test_output_y = tra_weightNN.predict(x_test_tensor)
test_accuracy = accuracy(test_output_y, y_test_tensor.data.numpy())

print('test accuracy is {}%'.format(100 * test_accuracy))

In [ ]:
"""
Different noise level
"""

noise_level = [0.1,0.2,0.4,0.6,0.8]
exp_summary = []

for level in noise_level:
    
    level_summary = {"noise_level": level, "std_test":0, "rewgt_test":0}       
        
    np.random.seed(int(time.time()))
    x_train = read_idx('data/train-images.idx3-ubyte')
    y_train = read_idx('data/train-labels.idx1-ubyte')
    x_test = read_idx('data/t10k-images.idx3-ubyte')
    y_test = read_idx('data/t10k-labels.idx1-ubyte')
    valid_idx = np.random.choice(range(60000), size=1000, replace=False)
    x_valid = x_train[valid_idx]
    y_valid = y_train[valid_idx]
    x_train = np.delete(x_train, valid_idx, axis=0)
    y_train = np.delete(y_train, valid_idx)

    y_train_noisy = mnist_noise(y_train, level) #apply noise level
 
    train_idx = np.arange(len(x_train))
    x_train = np.transpose(x_train,(2,1,0))
    x_valid = np.transpose(x_valid,(2,1,0))
    x_test = np.transpose(x_test,(2,1,0))
    x_train_tensor = torchvision.transforms.ToTensor()(x_train).unsqueeze(1)
    x_valid_tensor = torchvision.transforms.ToTensor()(x_valid).unsqueeze(1)
    x_test_tensor = torchvision.transforms.ToTensor()(x_test).unsqueeze(1)
    y_train_tensor = torch.from_numpy(y_train.astype(np.long))
    y_train_noisy_tensor = torch.from_numpy(y_train_noisy.astype(np.long))
    y_valid_tensor = torch.from_numpy(y_valid.astype(np.long))
    y_test_tensor = torch.from_numpy(y_test.astype(np.long))

    cnn = CNN()
    cnn.to(device)

    stand_trainNN = StandardTrainingNN(cnn,
                                       batch_size=64,
                                       num_iter=80,
                                       learning_rate=1e-3,
                                       early_stopping=5,
                                       device=device,
                                       iprint=0)
    stand_trainNN.fit(x_train_tensor, y_train_noisy_tensor, x_valid_tensor, y_valid_tensor, x_test_tensor, y_test_tensor)

    test_output_y = stand_trainNN.predict(x_test_tensor)
    test_accuracy = accuracy(test_output_y, y_test_tensor.data.numpy())
    level_summary['std_test'] = test_accuracy

    time.sleep(10)

    cnn = CNN()
    cnn.to(device)

    tra_weightNN = TrajectoryReweightNN(cnn,
                                        burnin=5,
                                        num_cluster=5,
                                        batch_size=64,
                                        num_iter=80,
                                        learning_rate=1e-3,
                                        early_stopping=5,
                                        device=device,
                                        traj_step = 3,
                                        iprint=0)
    tra_weightNN.fit(x_train_tensor, y_train_noisy_tensor, x_valid_tensor, y_valid_tensor,x_test_tensor, y_test_tensor)

    test_output_y = tra_weightNN.predict(x_test_tensor)
    test_accuracy = accuracy(test_output_y, y_test_tensor.data.numpy())
    level_summary['rewgt_test'] = test_accuracy

    time.sleep(10)
    
    exp_summary.append(level_summary)
    print('nosie level = {} finished running'.format(level))
    time.sleep(30)

nosie level = 0.1 finished running
nosie level = 0.2 finished running
nosie level = 0.4 finished running


In [ ]:
print(exp_summary)